In [ ]:
pip install imbalanced-learn

In [ ]:
import tensorflow as tf
import os
from fastai.vision.all import *
from nbdev.showdoc import *
import fastbook
fastbook.setup_book()
import random
from fastai.vision.augment import aug_transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from fastai.data.load import DataLoader
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

In [ ]:
bac_minus_folder = 'NEGATIVOS'
bac_plus_folder = 'POSITIVOS'

bac_minus_paths = get_image_files(bac_minus_folder)
bac_plus_paths = get_image_files(bac_plus_folder)

bac_minus_labels = [0] * len(bac_minus_paths)
bac_plus_labels = [1] * len(bac_plus_paths)

all_paths = bac_minus_paths + bac_plus_paths
#all_paths_array = np.array(all_paths).reshape(-1, 1)
all_paths_list = list(all_paths)


all_labels = bac_minus_labels + bac_plus_labels


print(len(bac_minus_paths))
print(len(bac_plus_paths))

print(len(bac_minus_labels))
print(len(bac_plus_labels))

print(len(all_paths))
print(len(all_labels))


In [ ]:
train_paths, test_paths, train_labels, test_labels = train_test_split(all_paths_list, all_labels, test_size=0.2, random_state=42, stratify=all_labels)

print(len(train_paths))
print(len(test_paths))

print(len(train_labels))
print(len(test_labels))

In [ ]:
batch_tfms = aug_transforms(flip_vert=False,do_flip=False)

train_df = pd.DataFrame({'paths': train_paths, 'labels': train_labels})
test_df = pd.DataFrame({'paths': test_paths, 'labels': test_labels})


In [ ]:
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_x=ColReader('paths'),
    get_y=ColReader('labels'),
    item_tfms=Resize(512),
    batch_tfms=batch_tfms,
    splitter=RandomSplitter(seed=42)
)

In [ ]:
dls = dblock.dataloaders(train_df)


In [ ]:
class RocAuc(Metric):
    def __init__(self): self._name = "roc_auc_score"
    def reset(self): self.preds, self.targets = [], []
    def accumulate(self, learn):
        preds = learn.pred.argmax(dim=-1).cpu().numpy()
        targets = learn.y.cpu().numpy()
        self.preds.extend(preds)
        self.targets.extend(targets)
    @property
    def value(self):
        if len(np.unique(self.targets)) == 1:
            return 0.0  # if only one class in targets, roc_auc_score can't be calculated
        return roc_auc_score(self.targets, self.preds)
    @property
    def name(self):
        return self._name
    
def get_weights(dls):
    classes = dls.vocab
    train_lbls = L(map(lambda x: classes[x[1]], dls.train_ds))
    label_counter = Counter(train_lbls)
    n_most_common_class = max(label_counter.values())
    print(f'Occurrences of the most common class {n_most_common_class}')
    weights = [n_most_common_class/v for k, v in label_counter.items() if v > 0]
    return weights

In [ ]:
vis_learn = cnn_learner(dls, resnet18, metrics=[accuracy, RocAuc()], pretrained=True)

In [ ]:
class_weights = get_weights(dls)

In [ ]:
weight_tensor = torch.FloatTensor(class_weights)

In [ ]:
vis_learn.loss_func = CrossEntropyLossFlat(weight=weight_tensor)

In [ ]:
vis_learn.fine_tune(15, freeze_epochs=15, 
                      cbs=[EarlyStoppingCallback(monitor='accuracy', patience=10), 
                           SaveModelCallback(monitor='accuracy', with_opt=True, fname='fastai2_VIS_weights')])

In [ ]:
train_losses = [x[0] for x in vis_learn.recorder.values]
valid_losses = [x[1] for x in vis_learn.recorder.values]
accuracies = [x[2] for x in vis_learn.recorder.values]
roc_auc_scores = [x[3] for x in vis_learn.recorder.values]

# Graficar las métricas
epochs = range(1, len(train_losses) + 1)
plt.figure(figsize=(12, 5))

# Pérdida de entrenamiento y validación
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'r', label='Train Loss')
plt.plot(epochs, valid_losses, 'b', label='Valid Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Precisión y ROC AUC
plt.subplot(1, 2, 2)
plt.plot(epochs, accuracies, 'g', label='Accuracy')
plt.plot(epochs, roc_auc_scores, 'm', label='ROC AUC')
plt.title('Accuracy and ROC AUC')
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.legend()

plt.show()


In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

# Confusion Matrix
preds, targs = vis_learn.get_preds()
# Verificar las formas de preds y targs
print(f'Predictions shape: {preds.shape}')
print(f'Targets shape: {targs.shape}')
if len(targs.shape) > 1 and targs.shape[1] == 1:
    targs = targs.squeeze(1)
print(f'Adjusted Targets shape: {targs.shape}')
cm = confusion_matrix(targs, preds.argmax(dim=1))
print('Confusion Matrix:')
print(cm)

plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=dls.vocab, yticklabels=dls.vocab)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

probs = preds[:, 1]
fpr, tpr, _ = roc_curve(targs, probs)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(10, 7))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()